# Let's try to redo the project with data from the past
---
After looking at the the past data, we will have to rescrape new data.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import pybaseball as pb
from pybaseball import cache

pd.set_option('display.max_columns', None)

In [2]:
# mlb2020 = pb.statcast(start_dt='2020-07-23', end_dt='2020-10-27')
# mlb2019 = pb.statcast(start_dt='2019-03-20', end_dt='2019-10-30')
# mlb2018 = pb.statcast(start_dt='2018-03-29', end_dt='2018-10-28')
# mlb2017 = pb.statcast(start_dt='2017-04-02', end_dt='2017-11-01')
# mlb2016 = pb.statcast(start_dt='2016-04-03', end_dt='2016-11-02')
# mlb2015 = pb.statcast(start_dt='2015-04-05', end_dt='2015-11-01')
print('Retrieval Complete')

This is a large query, it may take a moment to complete
Completed sub-query from 2020-07-23 to 2020-07-28
Completed sub-query from 2020-07-29 to 2020-08-03
Completed sub-query from 2020-08-04 to 2020-08-09
Completed sub-query from 2020-08-10 to 2020-08-15
Completed sub-query from 2020-08-16 to 2020-08-21
Completed sub-query from 2020-08-22 to 2020-08-27
Completed sub-query from 2020-08-28 to 2020-09-02
Completed sub-query from 2020-09-03 to 2020-09-08
Completed sub-query from 2020-09-09 to 2020-09-14
Completed sub-query from 2020-09-15 to 2020-09-20
Completed sub-query from 2020-09-21 to 2020-09-26
Completed sub-query from 2020-09-27 to 2020-10-02
Completed sub-query from 2020-10-03 to 2020-10-08
Completed sub-query from 2020-10-09 to 2020-10-14
Completed sub-query from 2020-10-15 to 2020-10-20
Completed sub-query from 2020-10-21 to 2020-10-26
Completed sub-query from 2020-10-27 to 2020-10-27
This is a large query, it may take a moment to complete
Completed sub-query from 2019-03-20 to

In [4]:
# mlb2020.to_csv('rescrape-past-seasons/2020mlb.csv', index=False)
# mlb2019.to_csv('rescrape-past-seasons/2019mlb.csv', index=False)
# mlb2018.to_csv('rescrape-past-seasons/2018mlb.csv', index=False)
# mlb2017.to_csv('rescrape-past-seasons/2017mlb.csv', index=False)
# mlb2016.to_csv('rescrape-past-seasons/2016mlb.csv', index=False)
# mlb2015.to_csv('rescrape-past-seasons/2015mlb.csv', index=False)

We will use the same cleaning and cleaning order to get the exact things we have completed.

In [7]:
mlb2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279660 entries, 0 to 279659
Data columns (total 93 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   index                            279660 non-null  int64         
 1   pitch_type                       279636 non-null  object        
 2   game_date                        279660 non-null  datetime64[ns]
 3   release_speed                    279636 non-null  float64       
 4   release_pos_x                    279636 non-null  float64       
 5   release_pos_z                    279636 non-null  float64       
 6   player_name                      279660 non-null  object        
 7   batter                           279660 non-null  float64       
 8   pitcher                          279660 non-null  float64       
 9   events                           70400 non-null   object        
 10  description                      279660 non-

In [8]:
mlb2015.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 712840 entries, 0 to 712839
Data columns (total 93 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   index                            712840 non-null  int64         
 1   pitch_type                       712173 non-null  object        
 2   game_date                        712840 non-null  datetime64[ns]
 3   release_speed                    712782 non-null  float64       
 4   release_pos_x                    712173 non-null  float64       
 5   release_pos_z                    712173 non-null  float64       
 6   player_name                      712840 non-null  object        
 7   batter                           712840 non-null  float64       
 8   pitcher                          712840 non-null  float64       
 9   events                           186657 non-null  object        
 10  description                      712840 non-

In [9]:
def full_clean(df):
    # Drops the columns we don't want
    df.drop(columns=['events','spin_dir','spin_rate_deprecated','break_angle_deprecated',
                     'break_length_deprecated','game_type','home_team','away_team',
                     'hit_location','game_year','on_3b','on_2b','on_1b',
                     'outs_when_up','inning','inning_topbot','hc_x','hc_y',
                     'tfs_deprecated','tfs_zulu_deprecated','umpire','sv_id',
                     'hit_distance_sc','launch_speed','launch_angle',
                     'game_pk', 'pitcher.1','fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 
                     'fielder_6', 'fielder_7', 'fielder_8', 'fielder_9',
                     'estimated_ba_using_speedangle', 'estimated_woba_using_speedangle',
                     'woba_value', 'woba_denom', 'babip_value', 'iso_value',
                     'launch_speed_angle', 'at_bat_number', 'pitch_number',
                     'home_score', 'away_score', 'bat_score', 'fld_score', 'post_away_score',
                     'post_home_score', 'post_bat_score', 'post_fld_score',
                     'if_fielding_alignment', 'of_fielding_alignment', 'spin_axis',
                     'delta_home_win_exp', 'delta_run_exp'], inplace=True)
    
    # Keeps rows that didn't result in getting hit by the bat
    df = df[df['bb_type'].isnull()]
    
    # We only want data that was a called strike or called ball
    df = df.loc[((df['description'] == 'ball') | (df['description'] == 'called_strike'))]
    
    # We don't need bb_type anymore
    df.drop(columns=['bb_type'], inplace=True)
    
    # At this point, any nulls in any row is a row that isn't needed
    df.dropna(inplace=True)
    
    # We also want to return to different versions of the df for EDA and modeling purposes
    df2 = df.loc[:, ~df.columns.isin(['index','game_date','player_name','batter','pitcher','description','des',
                'fielder_2','sz_top','sz_bot'])]

    df3 = df2.loc[:, ~df2.columns.isin(['zone','balls','strikes','effective_speed','release_extension',
                                        'release_pos_y','pitch_type'])]
    
    # Return 3 variables
    return df, df2, df3
    

In [10]:
mlb2020, mlb2020_unnecessary, train_test_2020 = full_clean(mlb2020)